In [16]:
!pip install lifetimes

In [17]:

!pip install sqlalchemy
!pip install mysql-connector-python

In [18]:
from sqlalchemy import create_engine
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions


In [19]:

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler


In [20]:

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


In [21]:

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

Read the data

In [22]:
df_ = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")
df = df_.copy()
df.shape

(541910, 8)

Before preprocessing data

In [23]:
df.describe().T
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")


After preprocessing

In [24]:
df.describe().T

df["TotalPrice"] = df["Quantity"] * df["Price"]
today_date = dt.datetime(2011, 12, 11)
df=df.loc[df["Country"]=="United Kingdom"]

df["TotalPrice"] = df["Quantity"] * df["Price"]
today_date = dt.datetime(2011, 12, 11)

Preparing the Lifetime Data Structure

#########################

recency: Time elapsed since the last purchase. Weekly. (based on the analysis day in cltv_df, personalized for each user here)\
T: Customer's age. Weekly. (time since the first purchase made before the analysis date)\
frequency: total number of repeated purchases (frequency>1)\
monetary_value: average earnings per purchase





In [25]:

cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})


In [27]:
cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

In [28]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

In [29]:
cltv_df = cltv_df[cltv_df["monetary"] > 0]
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,310.4400
12747.0000,366,370,11,381.4555
12748.0000,372,374,210,154.1924
12749.0000,209,214,5,815.5880
12820.0000,323,327,4,235.5850


In [30]:
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [31]:
cltv_df["frequency"] = cltv_df["frequency"].astype(int)

##############################################################
# BG-NBD Modelinin Kurulması
##############################################################

In [34]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2570 subjects, a: 0.12, alpha: 81.61, b: 2.51, r: 2.21>

In [35]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).head(10)

ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).sort_values(ascending=False).head(10)

cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

cltv_df.sort_values("expected_average_profit", ascending=False).head(20)



,recency,T,frequency,monetary,expected_average_profit
Customer ID,,,,,
14088.0000,312,323,13,3864.5546,3916.3832
18102.0000,366,368,60,3859.7391,3870.8387
15749.0000,97,333,3,3028.7800,3213.2739
14096.0000,97,102,17,3163.5882,3195.9719
17511.0000,370,374,31,2933.9431,2950.3463
17450.0000,359,368,46,2863.2749,2874.0455
13081.0000,359,372,11,2576.1255,2617.1708
16000.0000,0,3,3,2335.1200,2477.6641
16984.0000,41,131,2,2240.6750,2452.3234


Who are the top 10 customers we expect to make the most purchases within the next week?

In [36]:
bgf.conditional_expected_number_of_purchases_up_to_time(1,
                                                        cltv_df['frequency'],
                                                        cltv_df['recency'],
                                                        cltv_df['T']).sort_values(ascending=False).head(10)

cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])

cltv_df.sort_values("expected_purc_1_week", ascending=False).head(10)


,recency,T,frequency,monetary,expected_average_profit,expected_purc_1_week
Customer ID,,,,,,
12748.0000,372,374,210,154.1924,154.3360,0.4651
17841.0000,371,374,124,330.1344,330.6207,0.2764
13089.0000,366,370,97,606.3625,607.4725,0.2190
14606.0000,372,374,93,130.1390,130.4193,0.2086
15311.0000,373,374,91,667.7791,669.0782,0.2042
12971.0000,368,373,86,127.4859,127.7837,0.1933
13408.0000,371,374,62,453.5006,454.8161,0.1405
18102.0000,366,368,60,3859.7391,3870.8387,0.1380
13798.0000,370,373,57,650.9085,652.9340,0.1298


Who are the top 10 customers we expect to make the most purchases within the next month?

In [37]:

bgf.predict(4,
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sort_values(ascending=False).head(10)

cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

cltv_df.sort_values("expected_purc_1_month", ascending=False).head(10)

,recency,T,frequency,monetary,expected_average_profit,expected_purc_1_week,expected_purc_1_month
Customer ID,,,,,,,
12748.0000,372,374,210,154.1924,154.3360,0.4651,1.8595
17841.0000,371,374,124,330.1344,330.6207,0.2764,1.1051
13089.0000,366,370,97,606.3625,607.4725,0.2190,0.8758
14606.0000,372,374,93,130.1390,130.4193,0.2086,0.8339
15311.0000,373,374,91,667.7791,669.0782,0.2042,0.8167
12971.0000,368,373,86,127.4859,127.7837,0.1933,0.7730
13408.0000,371,374,62,453.5006,454.8161,0.1405,0.5619
18102.0000,366,368,60,3859.7391,3870.8387,0.1380,0.5518
13798.0000,370,373,57,650.9085,652.9340,0.1298,0.5192


##############################################################
# Calculating  with BG-NBD ve GG MODEL
##############################################################

In [38]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6,  # 6 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv.head()

cltv.shape
cltv = cltv.reset_index()
cltv.sort_values(by="clv", ascending=False).head(50)

cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(10)



,Customer ID,recency,T,frequency,monetary,expected_average_profit,expected_purc_1_week,expected_purc_1_month,clv
2486,18102.0000,366,368,60,3859.7391,3870.8387,0.1380,0.5518,13408.9716
589,14096.0000,97,102,17,3163.5882,3195.9719,0.1035,0.4136,8264.6652
2184,17450.0000,359,368,46,2863.2749,2874.0455,0.1065,0.4259,7684.6360
2213,17511.0000,370,374,31,2933.9431,2950.3463,0.0725,0.2900,5372.0305
1804,16684.0000,353,359,28,2209.9691,2223.6899,0.0682,0.2725,3803.6903
406,13694.0000,369,374,50,1275.7005,1280.1557,0.1141,0.4563,3667.2945
1485,16000.0000,0,3,3,2335.1200,2477.6641,0.0597,0.2382,3661.4203
587,14088.0000,312,323,13,3864.5546,3916.3832,0.0371,0.1485,3649.1961
1173,15311.0000,373,374,91,667.7791,669.0782,0.2042,0.8167,3430.3930
133,13089.0000,366,370,97,606.3625,607.4725,0.2190,0.8758,3339.8818


In [39]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_final[["clv"]])
cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])

In [40]:
cltv_final.sort_values(by="scaled_clv", ascending=False).head()

,Customer ID,recency,T,frequency,monetary,expected_average_profit,expected_purc_1_week,expected_purc_1_month,clv,scaled_clv
2486,18102.0000,366,368,60,3859.7391,3870.8387,0.1380,0.5518,13408.9716,1.0000
589,14096.0000,97,102,17,3163.5882,3195.9719,0.1035,0.4136,8264.6652,0.6164
2184,17450.0000,359,368,46,2863.2749,2874.0455,0.1065,0.4259,7684.6360,0.5731
2213,17511.0000,370,374,31,2933.9431,2950.3463,0.0725,0.2900,5372.0305,0.4006
1804,16684.0000,353,359,28,2209.9691,2223.6899,0.0682,0.2725,3803.6903,0.2837
